In [1]:

# from models.SqueezeNet import SqueezeNet
# from models.mtcnn import fixed_image_standardization
from models.utils import training
import torch
from torch import nn
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import numpy as np
import torchvision.models as models
import sys,os
sys.path.append(os.getcwd())



workers = 0 if os.name == 'nt' else 4

In [2]:
data_dir = './data'
batch_size = 64
epochs = 800
workers = 0 if os.name == 'nt' else 8

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))
print(torch.cuda.is_available())
print(torch.__version__)

Running on device: cuda:0
True
2.1.0+cu121


In [4]:
from models.mtcnn import MTCNN

mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

In [5]:
# preprocess = transforms.Compose([
#     transforms.Resize(256),
#     transforms.CenterCrop(224),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
# ])


# dataset = datasets.ImageFolder(data_dir, transform=transforms.Resize((512, 512)))
# dataset.samples = [
#     (p, p.replace(data_dir, data_dir + '_cropped'))
#         for p, _ in dataset.samples
# ]

# loader = DataLoader(
#     dataset,
#     num_workers=workers,
#     batch_size=batch_size,
#     collate_fn=training.collate_pil
# )

# for i, (x,y) in enumerate(loader):
#     # print(f"y is {y}")
#     mtcnn(x, save_path=y)
#     print('\rBatch {} of {}'.format(i + 1, len(loader)), end='')



preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

dataset = datasets.ImageFolder(data_dir+'_cropped', transform=preprocess)

In [6]:


squeeze = models.squeezenet1_1(pretrained=True)

# squeeze = squeezenet1_1(weights=SqueezeNet1_1_Weights.DEFAULT,progress=True, num_classes=499)

# state_dict = torch.utils.model_zoo.load_url('https://download.pytorch.org/models/squeezenet1_1-b8a52dc0.pth')
# squeeze = SqueezeNet(
# dropout=0.5, version='1_1')
# squeeze.load_state_dict(state_dict)

for param in squeeze.parameters():
    param.requires_grad = False


final_conv = nn.Conv2d(512, dataset.class_to_idx.__len__(), kernel_size=1)
nn.init.xavier_uniform_(final_conv.weight) # Xavier uniform initialization
# nn.init.normal_(final_conv.weight, mean=0.0, std=0.01) # Normal initialization
nn.init.constant_(final_conv.bias, 0) # Initialize bias with a constant
squeeze.classifier[1] = final_conv
squeeze = squeeze.to('cuda')


for param in squeeze.classifier[1].parameters():
    param.requires_grad = True


# 解冻模型的最后两个Fire模块
for param in squeeze.features[-1:].parameters():
    param.requires_grad = True


print(final_conv.weight.shape)



# print(squeeze.state_dict())

# print(squeeze.classifier[1].state_dict())
# print(squeeze.features[0])




torch.Size([499, 512, 1, 1])


e:\anaconda\envs\facenet\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
e:\anaconda\envs\facenet\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_1_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
optimizer = optim.Adam(squeeze.parameters(), lr=0.002)
scheduler = MultiStepLR(optimizer, [5, 10])


img_inds = np.arange(len(dataset))
np.random.shuffle(img_inds)
train_inds = img_inds[:int(0.8 * len(img_inds))]
val_inds = img_inds[int(0.8 * len(img_inds)):]

train_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(train_inds)
)
val_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(val_inds)
)

In [8]:
loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

In [9]:

writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10
print('\n\n初始化')
print('-' * 10)
squeeze.eval()
training.pass_epoch(
    squeeze, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True, device=device,
    writer=writer
   
)

for epoch in range(epochs):
    print('\n循环 {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)
    squeeze.train()
    training.pass_epoch(
        squeeze, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
 
    )

    squeeze.eval()
    training.pass_epoch(
        squeeze, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
     
    )
    

writer.close()



初始化
----------
